In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import requests

api_key = 	"5d8d2c64eca4c7ac5b30ba8e6dc22e0d"

# 起点和终点的经纬度
origin = '121.481578,31.14881'  # 上海纽约大学
destination = '121.478293,31.167869'  # 宿舍

driving_url = f'https://restapi.amap.com/v3/direction/driving?origin={origin}&destination={destination}&key={api_key}'
walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'
transit_url = f'https://restapi.amap.com/v3/direction/transit/integrated?origin={origin}&destination={destination}&city=上海&key={api_key}'

# 驾车路径规划
def get_driving_route():
    response = requests.get(driving_url)
    data = response.json()
    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        for path in paths:
            print(f"驾车路径总距离：{path['distance']}米")
            print(f"驾车预估时间：{int(path['duration']) / 60}分钟")
    else:
        print("驾车路径规划请求失败")

# 步行路径规划
def get_walking_route():
    response = requests.get(walking_url)
    data = response.json()
    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        for path in paths:
            print(f"步行路径总距离：{path['distance']}米")
            print(f"步行预估时间：{int(path['duration']) / 60}分钟")
    else:
        print("步行路径规划请求失败")


# 获取公共交通路径规划
def get_transit_route():
    response = requests.get(transit_url)
    data = response.json()
    if data['status'] == '1':
        transits = data['route']['transits']
        for transit in transits:
            print(f"公共交通总时间：{int(transit['duration']) / 60}分钟")
            print(f"总步行距离：{transit['walking_distance']}米")
            for segment in transit['segments']:
                bus = segment['bus']
                if bus['buslines']:
                    for busline in bus['buslines']:
                        print(f"公交线路：{busline['name']}, 预计时间：{int(busline['duration']) / 60}分钟")
    else:
        print("公共交通路径规划请求失败")

# 调用函数获取不同交通方式的路径规划和预估时间
get_driving_route()
get_walking_route()
get_transit_route()

In [ ]:
import requests
api_key = 	"5d8d2c64eca4c7ac5b30ba8e6dc22e0d"

origin = '121.43470926491898, 31.133361959372316'  # population point
destination = '121.4345520877142,31.131651522402894'  # park
walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'


# 步行路径规划
def get_walking_route():
    response = requests.get(walking_url)
    data = response.json()
    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        for path in paths:
            print(f"步行路径总距离：{path['distance']}米")
            print(f"步行预估时间：{int(path['duration']) / 60}分钟")
    else:
        print("步行路径规划请求失败")





get_walking_route()

步行路径总距离：4197米
步行预估时间：55.96666666666667分钟


In [ ]:
import requests
api_key = 	"5d8d2c64eca4c7ac5b30ba8e6dc22e0d"

origin = '121.43470926491898, 31.133361959372316'  # population point
destination = '121.4345520877142,31.131651522402894'  # park
walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'


# 步行路径规划
def get_walking_route():
    response = requests.get(walking_url)
    data = response.json()
    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        for path in paths:
            print(f"步行路径总距离：{path['distance']}米")
            print(f"步行预估时间：{int(path['duration']) / 60}分钟")
            steps = path['steps']
            for step in steps:
                if 'instruction' in step:
                    print(f"步行指示: {step['instruction']}")
        else:
            print("结束导航")
    else:
        print("步行路径规划请求失败")





get_walking_route()

步行路径总距离：462米
步行预估时间：6.166666666666667分钟
步行指示: 向西北步行40米右转
步行指示: 向东北步行43米右转
步行指示: 沿老沪闵路向东南步行276米右转
步行指示: 沿罗秀路向西步行103米到达目的地
No steps data available


In [ ]:
import pandas as pd
import requests

# Load the CSV file
file_path = '/content/gdrive/MyDrive/Capstone/data/OD/chunk_1.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Gaode API information
api_key = '5d8d2c64eca4c7ac5b30ba8e6dc22e0d'  # Replace with your actual Gaode API key
walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'

# Function to get walking route details
def get_walking_route(origin, destination):
    response = requests.get(walking_url)
    data = response.json()

    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        if paths:
            path = paths[0]  # Take the first path as the route
            distance = float(path['distance'])
            duration = float(path['duration']) / 60  # Convert seconds to minutes

            # Collect all instructions from steps
            instructions = [step['instruction'] for step in path['steps'] if 'instruction' in step]
            all_instructions = " -> ".join(instructions)

            return distance, duration, all_instructions
    return None, None, None

# Add new columns to the DataFrame
df['walking_distance'] = None
df['walking_duration'] = None
df['walking_instructions'] = None

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract origin and destination coordinates
    origin_point = row['origin_loc'].strip("POINT ()").split()
    destination_point = row['destination_loc'].strip("POINT ()").split()

    origin = f"{origin_point[1]},{origin_point[0]}"  # Gaode API uses "longitude,latitude"
    destination = f"{destination_point[1]},{destination_point[0]}"

    # Get walking route details from the API
    distance, duration, instructions = get_walking_route(origin, destination)

    # Update the DataFrame
    df.at[index, 'walking_distance'] = distance
    df.at[index, 'walking_duration'] = duration
    df.at[index, 'walking_instructions'] = instructions

print("Walking route details have been added and saved to 'updated_file_with_walking_routes.csv'.")
df


Walking route details have been added and saved to 'updated_file_with_walking_routes.csv'.


,origin_id,destination_id,distance_meters,population_weight,origin_loc,destination_loc,walking_distance,walking_duration,walking_instructions
0,19642,169,1599.375014,1,POINT (31.24744800045549 121.4400991566237),POINT (31.2465966074023 121.45589236103254),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
1,19642,170,1130.981103,1,POINT (31.24744800045549 121.4400991566237),POINT (31.242803543365604 121.44992703411208),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
2,19642,174,498.196497,1,POINT (31.24744800045549 121.4400991566237),POINT (31.245263739788992 121.43494518210703),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
3,19642,198,2404.930218,1,POINT (31.24744800045549 121.4400991566237),POINT (31.249437864719358 121.46297106704435),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
4,19642,222,1690.019821,1,POINT (31.24744800045549 121.4400991566237),POINT (31.25950593097475 121.44694433930889),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
...,...,...,...,...,...,...,...,...,...
9995,23366,5234,1946.850434,1,POINT (31.234871586477823 121.4598620928743),POINT (31.22054466419129 121.46640111292464),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
9996,23366,5651,2919.650191,1,POINT (31.234871586477823 121.4598620928743),POINT (31.218529661236683 121.44155435756056),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
9997,23366,6037,1795.776417,1,POINT (31.234871586477823 121.4598620928743),POINT (31.224994962160345 121.47302302161059),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...
9998,23366,6435,882.983523,1,POINT (31.234871586477823 121.4598620928743),POINT (31.235872944920363 121.46913939765386),462.0,6.166667,向西北步行40米右转 -> 向东北步行43米右转 -> 沿老沪闵路向东南步行276米右转 -...


In [ ]:
import pandas as pd
import os


# Load the original CSV file
file_path = '/content/gdrive/MyDrive/Capstone/data/OD/chunk_1.csv'  # Replace with your original file path
df = pd.read_csv(file_path)

# Determine the number of rows per chunk
chunk_size = 2000

# Specify the directory where you want to save the chunks
output_directory = "/content/gdrive/MyDrive/Capstone/data/OD"  # Your desired directory path


# Split the DataFrame into chunks and save each chunk as a new CSV
for i in range(0, len(df), chunk_size):
    # Create a chunk of the DataFrame
    chunk = df[i:i + chunk_size]

    # Save the chunk to a new CSV file
    chunk_file_name = os.path.join( output_directory, f'chunk_2000_{i // chunk_size + 1}.csv')
    chunk.to_csv(chunk_file_name, index=False)

    print(f"Saved {chunk_file_name}")

print("All chunks have been saved successfully.")


Saved /content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_1.csv
Saved /content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_2.csv
Saved /content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_3.csv
Saved /content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_4.csv
Saved /content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_5.csv
All chunks have been saved successfully.


In [ ]:
import pandas as pd
import requests

# Load the CSV file
file_path = '/content/gdrive/MyDrive/Capstone/data/OD/chunk_100_1.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Gaode API information
api_key = '5d8d2c64eca4c7ac5b30ba8e6dc22e0d'  # Replace with your actual Gaode API key

# Function to get walking route details
def get_walking_route(origin, destination):
    walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'
    response = requests.get(walking_url)
    data = response.json()

    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        if paths:
            path = paths[0]  # Take the first path as the route
            distance = float(path['distance'])
            duration = float(path['duration']) / 60  # Convert seconds to minutes

            # Collect all instructions from steps
            instructions = [step['instruction'] for step in path['steps'] if 'instruction' in step]
            all_instructions = " -> ".join(instructions)

            return distance, duration, all_instructions
    return None, None, None

# Add new columns to the DataFrame
df['walking_distance'] = None
df['walking_duration'] = None
df['walking_instructions'] = None

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract origin and destination coordinates
    origin_point = row['origin_loc'].strip("POINT ()").split()
    destination_point = row['destination_loc'].strip("POINT ()").split()

    origin = f"{origin_point[1]},{origin_point[0]}"  # Gaode API uses "longitude,latitude"
    destination = f"{destination_point[1]},{destination_point[0]}"
    print(origin,destination)

    # Get walking route details from the API
    distance, duration, instructions = get_walking_route(origin, destination)

    # Update the DataFrame
    df.at[index, 'walking_distance'] = distance
    df.at[index, 'walking_duration'] = duration
    df.at[index, 'walking_instructions'] = instructions



print("Walking route details have been added and saved to 'updated_file_with_walking_routes.csv'.")
df


121.4400991566237,31.24744800045549 121.45589236103254,31.2465966074023
121.4400991566237,31.24744800045549 121.44992703411208,31.242803543365604
121.4400991566237,31.24744800045549 121.43494518210703,31.245263739788992
121.4400991566237,31.24744800045549 121.46297106704435,31.249437864719358
121.4400991566237,31.24744800045549 121.44694433930889,31.25950593097475
121.4400991566237,31.24744800045549 121.43616086388103,31.262097890465967
121.4400991566237,31.24744800045549 121.4359169543954,31.25271077272682
121.4400991566237,31.24744800045549 121.42860532886695,31.250980500616155
121.4400991566237,31.24744800045549 121.42903786575403,31.249216202357378
121.4400991566237,31.24744800045549 121.4421817232718,31.223868064155745
121.4400991566237,31.24744800045549 121.4595345072101,31.24241695028837
121.4400991566237,31.24744800045549 121.42341159383054,31.246765860588464
121.4400991566237,31.24744800045549 121.46542136622779,31.256546638502137
121.4400991566237,31.24744800045549 121.425220

,origin_id,destination_id,distance_meters,population_weight,origin_loc,destination_loc,walking_distance,walking_duration,walking_instructions
0,19642,169,1599.375014,1,POINT (31.24744800045549 121.4400991566237),POINT (31.2465966074023 121.45589236103254),1788.0,23.833333,沿江宁路向东南步行482米左转 -> 沿长寿路步行143米直行 -> 沿长寿路辅路向东步行5...
1,19642,170,1130.981103,1,POINT (31.24744800045549 121.4400991566237),POINT (31.242803543365604 121.44992703411208),1220.0,16.266667,沿江宁路向东南步行119米左转 -> 沿澳门路步行699米右转 -> 沿西苏州路向东南步行4...
2,19642,174,498.196497,1,POINT (31.24744800045549 121.4400991566237),POINT (31.245263739788992 121.43494518210703),629.0,8.383333,向西步行179米向右前方行走 -> 向西北步行26米左转 -> 沿宜昌路向西南步行424米到...
3,19642,198,2404.930218,1,POINT (31.24744800045549 121.4400991566237),POINT (31.249437864719358 121.46297106704435),2918.0,38.9,沿江宁路向东南步行482米左转 -> 沿长寿路步行143米直行 -> 沿长寿路辅路向东步行5...
4,19642,222,1690.019821,1,POINT (31.24744800045549 121.4400991566237),POINT (31.25950593097475 121.44694433930889),3095.0,41.266667,沿江宁路向东南步行119米左转 -> 沿澳门路步行318米左转 -> 沿昌化路向北步行260...
...,...,...,...,...,...,...,...,...,...
95,19876,5319,2369.199307,1,POINT (31.246549685171377 121.43920084133958),POINT (31.231467836200213 121.42661108906422),2625.0,35.0,向东步行25米右转 -> 向东南步行37米右转 -> 向南步行21米右转 -> 步行16米右...
96,19876,9072,1476.457111,1,POINT (31.246549685171377 121.43920084133958),POINT (31.257663137532667 121.4333158073487),2195.0,29.266667,向西步行89米右转 -> 向北步行140米左转 -> 向西北步行26米右转 -> 沿宜昌路向...
97,19876,9073,2376.244320,1,POINT (31.246549685171377 121.43920084133958),POINT (31.259504382434525 121.42282924727834),2719.0,36.25,向西步行89米右转 -> 向北步行140米左转 -> 向西北步行26米右转 -> 沿宜昌路向...
98,19876,15321,2801.819093,1,POINT (31.246549685171377 121.43920084133958),POINT (31.22496410253977 121.43880913080065),None,None,None


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# Load the CSV file
file_path = '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_1.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Gaode API information
api_key = '5d8d2c64eca4c7ac5b30ba8e6dc22e0d'  # Replace with your actual Gaode API key

# Function to get walking route details
def get_walking_route(origin, destination):
    walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'
    response = requests.get(walking_url)
    data = response.json()

    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        if paths:
            path = paths[0]  # Take the first path as the route
            distance = float(path['distance'])
            duration = float(path['duration']) / 60  # Convert seconds to minutes

            # Collect all instructions from steps
            instructions = [step['instruction'] for step in path['steps'] if 'instruction' in step]
            all_instructions = " -> ".join(instructions)

            return distance, duration, all_instructions
    return None, None, None

# Add new columns to the DataFrame
df['walking_distance'] = None
df['walking_duration'] = None
df['walking_instructions'] = None


  # Iterate through each row in the DataFrame
for index, row in df.iterrows():
      # Extract origin and destination coordinates
      origin_point = row['origin_loc'].strip("POINT ()").split()
      destination_point = row['destination_loc'].strip("POINT ()").split()

      origin = f"{origin_point[1]},{origin_point[0]}"  # Gaode API uses "longitude,latitude"
      destination = f"{destination_point[1]},{destination_point[0]}"
      print(origin,destination)

      # Get walking route details from the API
      distance, duration, instructions = get_walking_route(origin, destination)

      # Update the DataFrame
      df.at[index, 'walking_distance'] = distance
      df.at[index, 'walking_duration'] = duration
      df.at[index, 'walking_instructions'] = instructions



print("Walking route details have been added and saved to 'updated_file_with_walking_routes.csv'.")
df

121.4400991566237,31.24744800045549 121.45589236103254,31.2465966074023
121.4400991566237,31.24744800045549 121.44992703411208,31.242803543365604
121.4400991566237,31.24744800045549 121.43494518210703,31.245263739788992
121.4400991566237,31.24744800045549 121.46297106704435,31.249437864719358
121.4400991566237,31.24744800045549 121.44694433930889,31.25950593097475
121.4400991566237,31.24744800045549 121.43616086388103,31.262097890465967
121.4400991566237,31.24744800045549 121.4359169543954,31.25271077272682
121.4400991566237,31.24744800045549 121.42860532886695,31.250980500616155
121.4400991566237,31.24744800045549 121.42903786575403,31.249216202357378
121.4400991566237,31.24744800045549 121.4421817232718,31.223868064155745
121.4400991566237,31.24744800045549 121.4595345072101,31.24241695028837
121.4400991566237,31.24744800045549 121.42341159383054,31.246765860588464
121.4400991566237,31.24744800045549 121.46542136622779,31.256546638502137
121.4400991566237,31.24744800045549 121.425220

,origin_id,destination_id,distance_meters,population_weight,origin_loc,destination_loc,walking_distance,walking_duration,walking_instructions
0,19642,169,1599.375014,1,POINT (31.24744800045549 121.4400991566237),POINT (31.2465966074023 121.45589236103254),1788.0,23.833333,沿江宁路向东南步行482米左转 -> 沿长寿路步行143米直行 -> 沿长寿路辅路向东步行5...
1,19642,170,1130.981103,1,POINT (31.24744800045549 121.4400991566237),POINT (31.242803543365604 121.44992703411208),1220.0,16.266667,沿江宁路向东南步行119米左转 -> 沿澳门路步行699米右转 -> 沿西苏州路向东南步行4...
2,19642,174,498.196497,1,POINT (31.24744800045549 121.4400991566237),POINT (31.245263739788992 121.43494518210703),629.0,8.383333,向西步行179米向右前方行走 -> 向西北步行26米左转 -> 沿宜昌路向西南步行424米到...
3,19642,198,2404.930218,1,POINT (31.24744800045549 121.4400991566237),POINT (31.249437864719358 121.46297106704435),2918.0,38.9,沿江宁路向东南步行482米左转 -> 沿长寿路步行143米直行 -> 沿长寿路辅路向东步行5...
4,19642,222,1690.019821,1,POINT (31.24744800045549 121.4400991566237),POINT (31.25950593097475 121.44694433930889),None,None,None
...,...,...,...,...,...,...,...,...,...
1995,21472,1374,2982.784290,1,POINT (31.241159793466664 121.45806546230608),POINT (31.230029001301702 121.48215124641509),3224.0,42.983333,向东北步行17米向左前方行走 -> 向西北步行10米右转 -> 沿恒通路向东北步行139米右...
1996,21472,2572,2469.800740,1,POINT (31.241159793466664 121.45806546230608),POINT (31.223676180413687 121.4675748673717),2476.0,33.016667,向西步行45米左转 -> 沿恒丰路辅路向东南步行252米向右前方行走 -> 向南步行29米左...
1997,21472,3358,1790.005864,1,POINT (31.241159793466664 121.45806546230608),POINT (31.230719598642676 121.4707004566506),None,None,None
1998,21472,3530,2132.476151,1,POINT (31.241159793466664 121.45806546230608),POINT (31.22746276178601 121.4705453265176),2440.0,32.533333,向西步行45米左转 -> 沿恒丰路辅路向东南步行252米向右前方行走 -> 向南步行29米左...


In [ ]:
# Save the updated DataFrame to a new CSV file
df.to_csv('/content/gdrive/MyDrive/Capstone/data/OD_time_distance_instruction_2000_1', index=False)

## PIPLine CODE

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# Load the CSV file
file_path = [
    '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_2.csv',
    '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_3.csv',
    '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_4.csv',
    '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_5.csv'
]

df = pd.read_csv(file_path)

# Gaode API information
api_key = '5d8d2c64eca4c7ac5b30ba8e6dc22e0d'  # Replace with your actual Gaode API key

df = pd.read_csv(file_path)
# Function to get walking route details
def get_walking_route(origin, destination):
    walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'
    response = requests.get(walking_url)
    data = response.json()

    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        if paths:
            path = paths[0]  # Take the first path as the route
            distance = float(path['distance'])
            duration = float(path['duration']) / 60  # Convert seconds to minutes

            # Collect all instructions from steps
            instructions = [step['instruction'] for step in path['steps'] if 'instruction' in step]
            all_instructions = " -> ".join(instructions)

            return distance, duration, all_instructions
    return None, None, None

# Add new columns to the DataFrame
df['walking_distance'] = None
df['walking_duration'] = None
df['walking_instructions'] = None


  # Iterate through each row in the DataFrame
for index, row in df.iterrows():
      # Extract origin and destination coordinates
      origin_point = row['origin_loc'].strip("POINT ()").split()
      destination_point = row['destination_loc'].strip("POINT ()").split()

      origin = f"{origin_point[1]},{origin_point[0]}"  # Gaode API uses "longitude,latitude"
      destination = f"{destination_point[1]},{destination_point[0]}"
      print(origin,destination)

      # Get walking route details from the API
      distance, duration, instructions = get_walking_route(origin, destination)

      # Update the DataFrame
      df.at[index, 'walking_distance'] = distance
      df.at[index, 'walking_duration'] = duration
      df.at[index, 'walking_instructions'] = instructions




# Save the updated DataFrame to a new CSV file
df.to_csv('/content/gdrive/MyDrive/Capstone/data/OD_time_distance_instruction_2000_1', index=False)
print("Walking route details have been added and saved to 'updated_file_with_walking_routes.csv'.")
df

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
import os
# Gaode API information
api_key = '5d8d2c64eca4c7ac5b30ba8e6dc22e0d'  # Replace with your actual Gaode API key

    # '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_2.csv',
    # '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_3.csv',
    # '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_4.csv',
    # '/content/gdrive/MyDrive/Capstone/data/OD/chunk_2000_5.csv'
    # "/content/gdrive/MyDrive/Capstone/data/OD/chunk_2.csv"
# Load the CSV file
file_paths = ["/content/gdrive/MyDrive/Capstone/data/OD/chunk_3.csv",
      ]

def get_walking_route(origin, destination):
    walking_url = f'https://restapi.amap.com/v3/direction/walking?origin={origin}&destination={destination}&key={api_key}'
    response = requests.get(walking_url)
    data = response.json()

    if data['status'] == '1':
        route = data['route']
        paths = route['paths']
        if paths:
            path = paths[0]  # Take the first path as the route
            distance = float(path['distance'])
            duration = float(path['duration']) / 60  # Convert seconds to minutes

            # Collect all instructions from steps
            instructions = [step['instruction'] for step in path['steps'] if 'instruction' in step]
            all_instructions = " -> ".join(instructions)

            return distance, duration, all_instructions
    return None, None, None

# Loop through each file in file_paths
for file_path in file_paths:
    # Read the individual CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Add new columns for walking route details
    df['walking_distance'] = None
    df['walking_duration'] = None
    df['walking_instructions'] = None



    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    # for index, row in df.iterrows():
      # Extract origin and destination coordinates
          origin_point = row['origin_loc'].strip("POINT ()").split()
          destination_point = row['destination_loc'].strip("POINT ()").split()

          origin = f"{origin_point[1]},{origin_point[0]}"  # Gaode API uses "longitude,latitude"
          destination = f"{destination_point[1]},{destination_point[0]}"
          #print(origin,destination)

          # Get walking route details from the API
          distance, duration, instructions = get_walking_route(origin, destination)

          # Update the DataFrame
          df.at[index, 'walking_distance'] = distance
          df.at[index, 'walking_duration'] = duration
          df.at[index, 'walking_instructions'] = instructions

    # Generate output file name based on input file name
    base_name = os.path.basename(file_path)  # Extract the file name (e.g., chunk_2000_2.csv)
    name_without_extension = os.path.splitext(base_name)[0]  # Remove extension
    output_file_name = f'/content/gdrive/MyDrive/Capstone/data/{name_without_extension}_processed.csv'

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file_name, index=False)
    print(f"Processed file saved as {output_file_name}")







Processing rows:  89%|████████▉ | 8914/10000 [3:02:42<21:32,  1.19s/it]